<a href="https://colab.research.google.com/github/fertorresfs/Pipeline-Automatizado-de-Limpeza-de-Dados/blob/main/pipeline_csnu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configuração, Bibliotecas e Funções de Pipeline

In [18]:
import pandas as pd
import numpy as np
import requests
import logging
from datetime import datetime
import re
from io import StringIO
import os

# --- 1. Configuração de Logging ---
LOG_FILE = '/content/drive/MyDrive/Pipelines/CSNU/log/pipeline_ofac.log'
os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)
logging.basicConfig(filename=LOG_FILE, level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s', force=True)
print(f"Configuração de logging concluída. O log será gravado em '{os.path.abspath(LOG_FILE)}'.")

# --- DEFINIÇÃO DO CABEÇALHO DO SDN.CSV (CRÍTICO) ---
# Nomes das 12 colunas do SDN.CSV da OFAC, conforme a documentação
OFAC_SDN_HEADERS = [
    'ent_num', 'SDN_Name', 'SDN_Type', 'Program', 'Title',
    'Call_Sign', 'Vess_type', 'Tonnage', 'GRT', 'Vess_flag',
    'Vess_owner', 'Remarks'
]


# ======================================================================
# FASE E & L (Extract & Load) - CORRIGIDA
# ======================================================================

def ingestao_csv_ofac(file_name='SDN.CSV'):
    """
    Extrai o arquivo CSV da lista de sanções da OFAC, ignorando linhas de metadados
    e aplicando o cabeçalho correto.
    """
    BASE_URL = "https://sanctionslistservice.ofac.treas.gov/api/download/"
    FULL_URL = f"{BASE_URL}{file_name}"

    try:
        logging.info(f"Iniciando requisição GET para download do arquivo: {file_name}")
        response = requests.get(FULL_URL, timeout=60)
        response.raise_for_status()

        csv_data = StringIO(response.text)

        # --- CORREÇÃO CRÍTICA: Definir header=None e skiprows=1 ---
        # 1. header=None: Informa ao Pandas para não usar a primeira linha como cabeçalho (já que são dados)
        # 2. names=OFAC_SDN_HEADERS: Aplica o cabeçalho definido manualmente
        # 3. skiprows=1: Tenta pular a primeira linha (que o log mostrou ser dados) se ela for de metadados
        df = pd.read_csv(csv_data,
                         na_values=['-0-'],
                         encoding='latin1',
                         skipinitialspace=True,
                         header=None, # Tenta ler sem cabeçalho automático
                         names=OFAC_SDN_HEADERS, # Define o cabeçalho manualmente
                         skiprows=1 # Pula a primeira linha, que é um registro completo (ex: 36, AEROCARIBBEAN...)
                        )
        # --------------------------------------------------------

        # A limpeza de colunas não é mais necessária, pois usamos 'names'

        logging.info(f"Dados do arquivo {file_name} carregados com sucesso. Linhas: {len(df)}")
        logging.info(f"Colunas Finais Ingeridas: {list(df.columns)}")
        return df

    except Exception as e:
        logging.error(f"Erro inesperado durante a ingestão: {e}")
        return None


# Funções de Transformação (Mantidas, pois os nomes de coluna estão CORRETOS agora)

def tratar_valores_ausentes(df, colunas_numericas, estrategia='remover_linhas'):
    df_out = df.copy()
    if estrategia == 'remover_linhas':
        colunas_criticas = ['ent_num', 'SDN_Name', 'Program']
        df_out.dropna(subset=[col for col in colunas_criticas if col in df_out.columns], inplace=True)
        logging.info(f"Linhas com nulos em colunas críticas removidas.")
    elif estrategia == 'preencher_media':
        for col in colunas_numericas:
            if col in df_out.columns and pd.api.types.is_numeric_dtype(df_out[col]):
                media = df_out[col].mean()
                df_out[col].fillna(media, inplace=True)
                logging.info(f"Valores ausentes da coluna '{col}' preenchidos com a média: {media:.2f}")
    return df_out

def validar_tipos(df, colunas_numericas, colunas_categoricas):
    df_out = df.copy()
    for col in colunas_numericas:
        if col in df_out.columns:
            df_out[col] = pd.to_numeric(df_out[col], errors='coerce')
            logging.info(f"Coluna '{col}' convertida para numérica.")
    for col in colunas_categoricas:
        if col in df_out.columns:
            df_out[col] = df_out[col].astype('category')
            logging.info(f"Coluna '{col}' convertida para categórica.")
    return df_out

def limpar_e_extrair_aliases(df):
    """Extrai os "Weak Aliases" (AKAs Fracos) da coluna 'Remarks'."""
    df_out = df.copy()
    coluna_name = 'SDN_Name'
    coluna_remarks = 'Remarks' # Nome de coluna correto agora

    if coluna_remarks not in df_out.columns or coluna_name not in df_out.columns:
        # Isso não deve mais acontecer
        logging.warning("Falha na identificação de colunas críticas, mesmo após renomeação manual.")
        df_out['SDN_Name_Clean'] = None
        df_out['Weak_Aliases_Clean'] = None
        return df_out

    try:
        logging.info("Iniciando limpeza de texto e extração de 'Weak Aliases'.")

        df_out['SDN_Name_Clean'] = df_out[coluna_name].str.lower().str.strip()

        # Regex busca strings entre aspas duplas, conforme a documentação OFAC para o CSV.
        raw_aliases = df_out[coluna_remarks].apply(
            lambda x: re.findall(r'"([^"]*)"', str(x)) if pd.notna(x) else []
        )

        df_out['Weak_Aliases_Clean'] = raw_aliases.apply(
             lambda aliases: [str(a).lower().strip() for a in aliases]
        )

        logging.info("Limpeza de texto e extração de 'Weak Aliases' concluídas.")
        return df_out

    except Exception as e:
        logging.error(f"Erro crítico na limpeza de texto/extração de aliases: {e}")
        df_out['SDN_Name_Clean'] = None
        df_out['Weak_Aliases_Clean'] = None
        return df_out

def detectar_outliers(df, colunas_numericas, metodo='iqr', limiar=1.5):
  df_out = df.copy()
  if metodo == 'iqr':
      for col in colunas_numericas:
        if col in df_out.columns and pd.api.types.is_numeric_dtype(df_out[col]):
            if col in ['ent_num']: continue
            Q1 = df_out[col].quantile(0.25)
            Q3 = df_out[col].quantile(0.75)
            IQR = Q3 - Q1
            limite_inferior = Q1 - (limiar * IQR)
            limite_superior = Q3 + (limiar * IQR)
            df_out = df_out[(df_out[col] >= limite_inferior) & (df_out[col] <= limite_superior)]
            logging.info(f"Outliers detectados e removidos (IQR) na coluna '{col}'.")
  return df_out

# ======================================================================
# VARIÁVEIS DE CONFIGURAÇÃO DO PIPELINE
# ======================================================================
COLUNAS_NUMERICAS_SDN = ['ent_num', 'Tonnage', 'GRT']
COLUNAS_CATEGORICAS_SDN = ['SDN_Type', 'Program', 'Vess_type', 'Vess_flag']

Configuração de logging concluída. O log será gravado em '/content/drive/MyDrive/Pipelines/CSNU/log/pipeline_ofac.log'.


#Execução do Pipeline ELT

In [19]:
# --- EXECUÇÃO DO PIPELINE ---
NOME_ARQUIVO_ENTRADA = 'SDN.CSV'
NOME_ARQUIVO_SAIDA = 'dados_limpos_ofac_sdn.csv'
COLUNAS_RELATORIO = ['ent_num', 'SDN_Name_Clean', 'Program', 'Tonnage', 'Weak_Aliases_Clean']

print(f"\n--- INICIANDO PIPELINE ELT/LIMPEZA DE DADOS: {NOME_ARQUIVO_ENTRADA} ---")

# ======================================================================
# PASSO 1: E (Extract) & L (Load)
# ======================================================================
df_dados_brutos = ingestao_csv_ofac(file_name=NOME_ARQUIVO_ENTRADA)

if df_dados_brutos is None:
    print("\n[ERRO FATAL] Falha na Ingestão. Consulte o log para detalhes.")
else:
    print(f"\n[SUCESSO] Dados brutos carregados. Linhas: {len(df_dados_brutos)}")

    df_trabalho = df_dados_brutos.copy()

    # ======================================================================
    # PASSO 2: T (Transform) - Limpeza e Padronização
    # ======================================================================

    print("-> [2.1] VALIDANDO E CONVERTENDO TIPOS...")
    df_trabalho = validar_tipos(df_trabalho, COLUNAS_NUMERICAS_SDN, COLUNAS_CATEGORICAS_SDN)

    print("-> [2.2] TRATANDO VALORES AUSENTES...")
    df_trabalho = tratar_valores_ausentes(df_trabalho, COLUNAS_NUMERICAS_SDN, estrategia='remover_linhas')
    df_trabalho = tratar_valores_ausentes(df_trabalho, ['Tonnage', 'GRT'], estrategia='preencher_media')

    # Esta função deve encontrar as colunas
    print("-> [2.3] EXTRAINDO E PADRONIZANDO ALIASES...")
    df_trabalho = limpar_e_extrair_aliases(df_trabalho)

    print("-> [2.4] DETECTANDO OUTLIERS (IQR)...")
    df_trabalho = detectar_outliers(df_trabalho, ['Tonnage', 'GRT'], metodo='iqr', limiar=1.5)

    # ======================================================================
    # PASSO 3: T (Transform) - Saída e Relatório
    # ======================================================================

    print("\n--- RELATÓRIO DE LIMPEZA ---")
    print(f"Linhas Originais: {len(df_dados_brutos)}")
    print(f"Linhas Finais (após limpeza): {len(df_trabalho)}")
    print(f"Linhas Descartadas: {len(df_dados_brutos) - len(df_trabalho)}")
    print("-" * 30)

    colunas_existentes = [col for col in COLUNAS_RELATORIO if col in df_trabalho.columns]

    print("AMOSTRA DO DATAFRAME LIMPO:")
    if colunas_existentes:
        print(df_trabalho[colunas_existentes].head())
    else:
        # Se a amostra falhar, pelo menos mostra o cabeçalho final para novo diagnóstico
        print("Amostra de colunas de relatório falhou. Cabeçalho final do DataFrame:")
        print(df_trabalho.head())

    df_trabalho.to_csv(NOME_ARQUIVO_SAIDA, index=False)
    logging.info(f"Pipeline concluído. Dados limpos salvos em '{NOME_ARQUIVO_SAIDA}' (Linhas: {len(df_trabalho)}).")
    print(f"\n[SUCESSO] Dados finais salvos em '{NOME_ARQUIVO_SAIDA}'.")


--- INICIANDO PIPELINE ELT/LIMPEZA DE DADOS: SDN.CSV ---

[SUCESSO] Dados brutos carregados. Linhas: 18256
-> [2.1] VALIDANDO E CONVERTENDO TIPOS...
-> [2.2] TRATANDO VALORES AUSENTES...
-> [2.3] EXTRAINDO E PADRONIZANDO ALIASES...
-> [2.4] DETECTANDO OUTLIERS (IQR)...

--- RELATÓRIO DE LIMPEZA ---
Linhas Originais: 18256
Linhas Finais (após limpeza): 18152
Linhas Descartadas: 104
------------------------------
AMOSTRA DO DATAFRAME LIMPO:
   ent_num             SDN_Name_Clean Program     Tonnage Weak_Aliases_Clean
0    173.0  anglo-caribbean co., ltd.    CUBA  909.285714                 []
1    306.0     banco nacional de cuba    CUBA  909.285714                 []
2    424.0         boutique la maison    CUBA  909.285714                 []
3    475.0               casa de cuba    CUBA  909.285714                 []
4    480.0               cecoex, s.a.    CUBA  909.285714                 []


/tmp/ipython-input-1529491925.py:82: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_out[col].fillna(media, inplace=True)



[SUCESSO] Dados finais salvos em 'dados_limpos_ofac_sdn.csv'.


#Configuração e Funções de Pipeline (Expansão para Múltiplos Arquivos)

In [21]:
import pandas as pd
import numpy as np
import requests
import logging
from datetime import datetime
import re
from io import StringIO
import os

# --- 1. Configuração de Logging ---
LOG_FILE = '/content/drive/MyDrive/Pipelines/CSNU/log/pipeline_ofac_mult.log'
os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)
logging.basicConfig(filename=LOG_FILE, level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s', force=True)
print(f"Configuração de logging concluída. O log será gravado em '{os.path.abspath(LOG_FILE)}'.")

# --- DEFINIÇÃO DOS CABEÇALHOS (CRÍTICO) ---
# SDN.CSV (12 colunas)
OFAC_SDN_HEADERS = [
    'ent_num', 'SDN_Name', 'SDN_Type', 'Program', 'Title',
    'Call_Sign', 'Vess_type', 'Tonnage', 'GRT', 'Vess_flag',
    'Vess_owner', 'Remarks'
]
# ADD.CSV (6 colunas)
OFAC_ADD_HEADERS = [
    'Ent_num', 'Add_num', 'Address', 'City_State_Province_PostalCode',
    'Country', 'Add_remarks'
]
# ALT.CSV (5 colunas)
OFAC_ALT_HEADERS = [
    'ent_num', 'alt_num', 'alt_type', 'alt_name', 'alt_remarks'
]


# ======================================================================
# FASE E & L (Extract & Load)
# ======================================================================

def ingestao_csv_ofac(file_name):
    """
    Extrai o arquivo CSV da OFAC e aplica o cabeçalho e as regras de leitura corretas.
    """
    if file_name == 'SDN.CSV':
        headers = OFAC_SDN_HEADERS
    elif file_name == 'ADD.CSV':
        headers = OFAC_ADD_HEADERS
    elif file_name == 'ALT.CSV':
        headers = OFAC_ALT_HEADERS
    else:
        logging.error(f"Nome de arquivo desconhecido: {file_name}")
        return None

    BASE_URL = "https://sanctionslistservice.ofac.treas.gov/api/download/"
    FULL_URL = f"{BASE_URL}{file_name}"

    try:
        logging.info(f"Iniciando requisição GET para download do arquivo: {file_name}")
        response = requests.get(FULL_URL, timeout=60)
        response.raise_for_status()

        csv_data = StringIO(response.text)

        # Leitura com skiprows=1 para ignorar a primeira linha (dados de exemplo/metadados)
        df = pd.read_csv(csv_data,
                         na_values=['-0-'],
                         encoding='latin1',
                         skipinitialspace=True,
                         header=None,
                         names=headers,
                         skiprows=1
                        )

        # Limpa o campo de ligação do ADD.CSV
        if file_name == 'ADD.CSV':
             df.rename(columns={'Ent_num': 'ent_num'}, inplace=True)

        logging.info(f"Dados do arquivo {file_name} carregados com sucesso. Linhas: {len(df)}")
        return df

    except Exception as e:
        logging.error(f"Erro inesperado durante a ingestão do {file_name}: {e}")
        return None


# ======================================================================
# FASE T (Transformação/Relatório)
# ======================================================================

def gerar_relatorio_qualidade(df_final, df_sdn_orig):
    """
    Gera um relatório detalhado sobre a qualidade da limpeza e a consolidação dos dados.

    Args:
        df_final (pd.DataFrame): DataFrame consolidado (SDN + ADD + ALT).
        df_sdn_orig (pd.DataFrame): DataFrame SDN após a ingestão, antes da limpeza.

    Returns:
        str: Relatório formatado em texto.
    """

    # 1. Metadados de Linhas
    linhas_originais = len(df_sdn_orig)
    linhas_finais = df_final['ent_num'].nunique() # Contagem de entidades únicas no final
    linhas_totais_mescladas = len(df_final)

    # 2. Qualidade da Limpeza
    # Nota: Assumimos que a coluna 'Tonnage' foi preenchida com a média
    total_nulos_tonnage_preenchido = df_sdn_orig['Tonnage'].isna().sum()

    # 3. Métricas de Consolidação
    entidades_sdn = df_final['ent_num'].nunique()
    aliases_fortes_agregados = df_final['Strong_Aliases_Clean'].apply(len).sum()
    aliases_fracos_agregados = df_final['Weak_Aliases_Clean'].apply(len).sum()

    # Contagem média de entidades x endereços (para entender a granularidade)
    media_enderecos_por_entidade = linhas_totais_mescladas / entidades_sdn

    relatorio = f"""
======================================================================
         RELATÓRIO DE QUALIDADE E CONSOLIDAÇÃO DO PIPELINE OFAC
======================================================================
1. ESTATÍSTICAS DE INGESTÃO E LINHAS
----------------------------------------------------------------------
Total de Entidades SDN (Antes da Limpeza): {linhas_originais}
Total de Entidades SDN Únicas (Após a Limpeza): {linhas_finais}
Total de Linhas no Dataset Consolidado (Incluindo Endereços): {linhas_totais_mescladas}
Linhas Descartadas (Nulos críticos/Outliers): {linhas_originais - linhas_finais}

2. QUALIDADE DE DADOS E TRANFORMACÃO
----------------------------------------------------------------------
Nº de Entidades com Aliases Fracos (Weak Aliases): {df_final[df_final['Weak_Aliases_Clean'].apply(len) > 0]['ent_num'].nunique()}
Total de Aliases Fracos (Agregado): {aliases_fracos_agregados}
Total de Aliases Fortes (Agregado - do ALT.CSV): {aliases_fortes_agregados}
Nº de Nulos em Tonnage PREENCHIDOS com a Média: {total_nulos_tonnage_preenchido}
Colunas Categóricas/Numéricas Validada: SDN_Type, Program, ent_num, Tonnage, GRT

3. ESTATÍSTICAS DE RELACIONAMENTO (Chave: ent_num)
----------------------------------------------------------------------
Média de Endereços por Entidade SDN: {media_enderecos_por_entidade:.2f}
"""
    logging.info("Relatório de Qualidade de Dados Gerado.")
    return relatorio

def validar_tipos(df, colunas_numericas, colunas_categoricas):
    """Garante que as colunas estejam no tipo de dado esperado."""
    df_out = df.copy()
    for col in colunas_numericas:
        if col in df_out.columns:
            df_out[col] = pd.to_numeric(df_out[col], errors='coerce')
            logging.info(f"Coluna '{col}' convertida para numérica.")
    for col in colunas_categoricas:
        if col in df_out.columns:
            df_out[col] = df_out[col].astype('category')
            logging.info(f"Coluna '{col}' convertida para categórica.")
    return df_out

def tratar_valores_ausentes(df, colunas_numericas, estrategia='remover_linhas'):
    """Trata valores ausentes."""
    df_out = df.copy()
    if estrategia == 'remover_linhas':
        # Remoção crítica em chaves de ligação e campos de identificação
        colunas_criticas = ['ent_num', 'SDN_Name', 'Program']
        df_out.dropna(subset=[col for col in colunas_criticas if col in df_out.columns], inplace=True)
        logging.info(f"Linhas com nulos em colunas críticas ({colunas_criticas}) removidas.")
    elif estrategia == 'preencher_media':
        for col in colunas_numericas:
            if col in df_out.columns and pd.api.types.is_numeric_dtype(df_out[col]):
                media = df_out[col].mean()
                df_out[col].fillna(media, inplace=True)
                logging.info(f"Valores ausentes da coluna '{col}' preenchidos com a média: {media:.2f}")
    return df_out

def limpar_e_extrair_aliases(df_sdn):
    """Extrai os "Weak Aliases" da coluna 'Remarks' do SDN.CSV."""
    df_out = df_sdn.copy()
    coluna_name = 'SDN_Name'
    coluna_remarks = 'Remarks'

    if coluna_remarks not in df_out.columns or coluna_name not in df_out.columns:
        logging.warning("Colunas SDN_Name ou Remarks não encontradas para extração de aliases fracos.")
        return df_out

    try:
        df_out['SDN_Name_Clean'] = df_out[coluna_name].str.lower().str.strip()

        # Regex busca strings entre aspas duplas na coluna Remarks
        raw_aliases = df_out[coluna_remarks].apply(
            lambda x: re.findall(r'"([^"]*)"', str(x)) if pd.notna(x) else []
        )

        df_out['Weak_Aliases_Clean'] = raw_aliases.apply(
             lambda aliases: [str(a).lower().strip() for a in aliases]
        )
        logging.info("Limpeza de texto e extração de 'Weak Aliases' (Fracos) concluídas.")
        return df_out

    except Exception as e:
        logging.error(f"Erro crítico na limpeza de texto/extração de aliases: {e}")
        return df_out

def mesclar_dados(df_sdn, df_add, df_alt):
    """
    Mescla (JOIN) os DataFrames SDN, Endereços e Aliases Forte.
    """
    logging.info("Iniciando mesclagem dos DataFrames (SDN, Endereços, Aliases).")

    # 1. Mescla SDN e Endereços (ADD)
    # Uma entidade SDN pode ter vários endereços, então mesclamos e agregamos.
    df_merged = pd.merge(df_sdn, df_add, on='ent_num', how='left', suffixes=('_sdn', '_add'))

    # 2. Mescla o resultado com Aliases (ALT)
    # Uma entidade SDN pode ter vários aliases fortes. Agrupamos antes de mesclar.
    df_alt_grouped = df_alt.groupby('ent_num')['alt_name'].apply(list).reset_index(name='Strong_Aliases_Clean')

    df_final = pd.merge(df_merged, df_alt_grouped, on='ent_num', how='left')

    # Preenche a coluna de aliases fortes com lista vazia onde não há match
    df_final['Strong_Aliases_Clean'].fillna(pd.Series([[]] * len(df_final)), inplace=True)

    logging.info(f"Mesclagem concluída. DataFrame final tem {len(df_final)} linhas.")
    return df_final


# ======================================================================
# VARIÁVEIS DE CONFIGURAÇÃO DO PIPELINE
# ======================================================================
COLUNAS_NUMERICAS_SDN = ['ent_num', 'Tonnage', 'GRT']
COLUNAS_CATEGORICAS_SDN = ['SDN_Type', 'Program', 'Vess_type', 'Vess_flag']

Configuração de logging concluída. O log será gravado em '/content/drive/MyDrive/Pipelines/CSNU/log/pipeline_ofac_mult.log'.


#Execução do Pipeline ELT de Mesclagem

In [22]:
# --- EXECUÇÃO DO PIPELINE EXPANDIDO ---
NOME_ARQUIVO_SAIDA = 'dados_conformidade_ofac_consolidado.csv'

print("\n--- INICIANDO PIPELINE DE CONFORMIDADE (SDN + ADD + ALT) ---")

# ======================================================================
# PASSO 1: E (Extract) & L (Load) dos Três Arquivos
# ======================================================================
df_sdn = ingestao_csv_ofac('SDN.CSV')
df_add = ingestao_csv_ofac('ADD.CSV')
df_alt = ingestao_csv_ofac('ALT.CSV')

if df_sdn is None or df_add is None or df_alt is None:
    print("\n[ERRO FATAL] Falha na ingestão de um ou mais arquivos. Abortando pipeline.")
else:
    print("\n[SUCESSO] Todos os arquivos principais carregados.")

    # ======================================================================
    # PASSO 2: T (Transform) - Preparação e Limpeza do SDN
    # ======================================================================

    df_sdn_trabalho = df_sdn.copy()

    print("-> [2.1] VALIDANDO E CONVERTENDO TIPOS (SDN)...")
    df_sdn_trabalho = validar_tipos(df_sdn_trabalho, COLUNAS_NUMERICAS_SDN, COLUNAS_CATEGORICAS_SDN)

    print("-> [2.2] TRATANDO VALORES AUSENTES (SDN)...")
    df_sdn_trabalho = tratar_valores_ausentes(df_sdn_trabalho, COLUNAS_NUMERICAS_SDN, estrategia='remover_linhas')
    df_sdn_trabalho = tratar_valores_ausentes(df_sdn_trabalho, ['Tonnage', 'GRT'], estrategia='preencher_media')

    print("-> [2.3] EXTRAINDO E PADRONIZANDO ALIASES FRACOS (SDN)...")
    df_sdn_trabalho = limpar_e_extrair_aliases(df_sdn_trabalho)

    print("-> [2.4] DETECTANDO OUTLIERS (SDN)...")
    # Aplica-se apenas aos dados numéricos não-ID
    df_sdn_trabalho = detectar_outliers(df_sdn_trabalho, ['Tonnage', 'GRT'], metodo='iqr', limiar=1.5)

    # ======================================================================
    # PASSO 3: T (Transform) - Limpeza Auxiliar e Mesclagem
    # ======================================================================

    print("-> [3.1] LIMPEZA E PREPARAÇÃO DOS ARQUIVOS AUXILIARES (ADD/ALT)...")
    # Limpeza básica em ADD/ALT (garante ID numérico para join)
    df_add['ent_num'] = pd.to_numeric(df_add['ent_num'], errors='coerce')
    df_alt['ent_num'] = pd.to_numeric(df_alt['ent_num'], errors='coerce')
    df_add.dropna(subset=['ent_num'], inplace=True)
    df_alt.dropna(subset=['ent_num'], inplace=True)

    print("-> [3.2] MESCLAGEM DOS DADOS (SDN + ADD + ALT)...")
    df_final = mesclar_dados(df_sdn_trabalho, df_add, df_alt)

    # ======================================================================
    # PASSO 4: SAÍDA E RELATÓRIO DE QUALIDADE DE DADOS
    # ======================================================================

    if df_sdn is not None and df_final is not None:
      print("\n--- INICIANDO GERAÇÃO DO RELATÓRIO DE QUALIDADE DE DADOS ---")

      # É necessário passar o DF SDN ANTES da limpeza de nulos críticos
      # para obter a contagem correta de linhas originais.
      df_sdn_bruto_para_relatorio = ingestao_csv_ofac('SDN.CSV')

      relatorio = gerar_relatorio_qualidade(df_final, df_sdn_bruto_para_relatorio)

      print(relatorio)

      print("\n--- RELATÓRIO DE CONSOLIDAÇÃO ---")
      print(f"Entidades SDN Originais: {len(df_sdn)}")
      print(f"Linhas Finais (Entidade x Endereço x Alias): {len(df_final)}")
      print("-" * 30)

      print("AMOSTRA DO DATAFRAME CONSOLIDADO (Chaves de Conformidade):")
      df_colunas_conformidade = ['ent_num', 'SDN_Name_Clean', 'Program', 'Tonnage', 'Country', 'Weak_Aliases_Clean', 'Strong_Aliases_Clean']
      print(df_final[df_final.columns.intersection(df_colunas_conformidade)].head())

      df_final.to_csv(NOME_ARQUIVO_SAIDA, index=False)
      logging.info(f"Pipeline concluído. Dados consolidados salvos em '{NOME_ARQUIVO_SAIDA}' (Linhas: {len(df_final)}).")
      print(f"\n[SUCESSO] Base de Dados de Conformidade consolidada salva em '{NOME_ARQUIVO_SAIDA}'.")


--- INICIANDO PIPELINE DE CONFORMIDADE (SDN + ADD + ALT) ---

[SUCESSO] Todos os arquivos principais carregados.
-> [2.1] VALIDANDO E CONVERTENDO TIPOS (SDN)...
-> [2.2] TRATANDO VALORES AUSENTES (SDN)...
-> [2.3] EXTRAINDO E PADRONIZANDO ALIASES FRACOS (SDN)...
-> [2.4] DETECTANDO OUTLIERS (SDN)...


/tmp/ipython-input-3651638275.py:114: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_out[col].fillna(media, inplace=True)


-> [3.1] LIMPEZA E PREPARAÇÃO DOS ARQUIVOS AUXILIARES (ADD/ALT)...
-> [3.2] MESCLAGEM DOS DADOS (SDN + ADD + ALT)...

--- RELATÓRIO DE CONSOLIDAÇÃO ---
Entidades SDN Originais: 18256
Linhas Finais (Entidade x Endereço x Alias): 23888
------------------------------
AMOSTRA DO DATAFRAME CONSOLIDADO (Chaves de Conformidade):


/tmp/ipython-input-3651638275.py:163: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['Strong_Aliases_Clean'].fillna(pd.Series([[]] * len(df_final)), inplace=True)


   ent_num Program     Tonnage             SDN_Name_Clean Weak_Aliases_Clean  \
0    173.0    CUBA  909.285714  anglo-caribbean co., ltd.                 []   
1    306.0    CUBA  909.285714     banco nacional de cuba                 []   
2    306.0    CUBA  909.285714     banco nacional de cuba                 []   
3    306.0    CUBA  909.285714     banco nacional de cuba                 []   
4    306.0    CUBA  909.285714     banco nacional de cuba                 []   

          Country     Strong_Aliases_Clean  
0  United Kingdom            [AVIA IMPORT]  
1     Switzerland  [NATIONAL BANK OF CUBA]  
2           Spain  [NATIONAL BANK OF CUBA]  
3           Japan  [NATIONAL BANK OF CUBA]  
4          Panama  [NATIONAL BANK OF CUBA]  

[SUCESSO] Base de Dados de Conformidade consolidada salva em 'dados_conformidade_ofac_consolidado.csv'.
